In [1]:
import sys
sys.path.append("../")

In [2]:
import json
from pprint import pprint as print_
from collections import OrderedDict
import numpy as np
from numpy.random import RandomState
from tqdm import tqdm_notebook as tqdm

from isanlp_srl_framebank.pipeline_default import PipelineDefault

In [3]:
random_seed = 41

In [4]:
roleset44 = {
 'содержание высказывания',
 'говорящий',
 'субъект социального отношения',
 'субъект психологического состояния',
 'содержание действия',
 'агенс',
 'тема',
 'конечная точка',
 'сфера',
 'контрагент',
 'субъект перемещения',
 'причина',
 'субъект поведения',
 'ситуация в фокусе',
 'исходный посессор',
 'субъект физиологической реакции',
 'адресат',
 'пациенс',
 'срок',
 'источник звука',
 'место',
 'признак',
 'потенциальная угроза',
 'субъект ментального состояния',
 'конечный посессор',
 'результат',
 'стимул',
 'субъект восприятия',
 'эффектор',
 'траектория',
 'содержание мысли',
 'пациенс перемещения',
 'каузатор',
 'предмет высказывания',
 'начальная точка',
 'способ',
 'пациенс социального отношения',
 'статус',
 'предмет мысли',
 'цель',
 'потенциальный пациенс',
 'контрагент социального отношения',
 'эталон',
 'признак действия'}

In [5]:
ARGUMENT_POSTAGS = {
        'NOUN',
        'PRON',
        'ADJ',
        'PROPN'
    }

In [6]:
def get_roles_pred(lemma, role_annot, part_id):
    ann_sent = role_annot[part_id]
    predicates = {}
    arguments = {}
    for event in ann_sent:
        predicate = {
            'lemma': lemma[part_id][event.pred[0]],
        }
        predicates[event.pred[0]] = predicate
        arguments[event.pred[0]] = []
        for arg in event.args:
            argument = {
                'tag': arg.tag,
                'lemma': lemma[part_id][arg.begin],
                'idx': arg.begin
            }
            arguments[event.pred[0]].append(argument)
            
    return predicates, arguments


def get_example(corpus, ex_number, part_id):
    words = []
    for obj in corpus[ex_number][1][part_id]:
        word = obj['form']
        for symbol in ':;,.!?':
            word = word.replace(' ' + symbol, symbol)
        words.append(word)
        
    return ' '.join(words)


def get_roles_true(annot, corpus, ex_number, part_id):
    predicates = {}
    arguments = {}
    postags = [item for sublist in annot['postag'] for item in sublist]
    for i, obj in enumerate(corpus[ex_number][1][part_id]):
        if 'rank' in obj:
            if obj['rank'] == 'Предикат':
                predicate = {
                    'lemma': obj['lemma']
                }
                predicates[i] = predicate
            else:
                if 'lemma' not in obj:
                    argument = {
                        'lemma': obj['form'],
                        'tag': obj['rolepred1'],
                        'idx': i
                    }
                else:
                    argument = {
                        'lemma': obj['lemma'],
                        'tag': obj['rolepred1'],
                        'idx': i
                    }
                
                argument['postag'] = postags[argument['idx']]
                pred_id = obj['fillpred']
                if pred_id not in arguments.keys():
                    arguments[pred_id] = []
                arguments[pred_id].append(argument)
                
    return predicates, arguments

In [18]:
def random_predictions(corpus, ppl, n_samples=100):
    np.random.seed(random_seed)
    samples_idxs = np.random.choice(len(corpus), size=n_samples)
    
    texts = [get_example(corpus, ex_num, 0) for ex_num in samples_idxs]
    annotations = [ppl(text) for text in tqdm(texts, desc='Analyzing texts')]
    pred_roles = [get_roles_pred(res['lemma'], res['srl'], 0) for res in annotations]
    
    true_roles = [get_roles_true(annotations[i], corpus, ex_num, 0) for i, ex_num in enumerate(samples_idxs)]
    
    
    repl_roles = {
        'агенс - субъект восприятия' : 'субъект восприятия',
        'агенс - субъект ментального состояния' : 'субъект ментального состояния',
        'результат / цель' : 'результат',
        'место - пациенс' : 'место',
        'говорящий - субъект психологического состояния' : 'субъект психологического состояния'
    }
    
    for role, val in repl_roles.items():
        for pair in true_roles:
            for _, args in pair[1].items():
                for arg in args:
                    arg['tag'] = arg['tag'].replace(role, val)
                    
    return true_roles, pred_roles, texts


def compute_metrics(y_pred, y_true, report_to=sys.stdout, roleset=roleset44, idxmatching=False):
    true_positive = 0
    condition_positive = 0
    predicted_condition_positive = 0
    error_examples = []
    
    print_func = lambda x: print(x, file=report_to)
    
    for i, (true_predicates, true_arguments) in enumerate(y_true):
        print_func(f"Inspecting example {i}")
        print_func(f"Expecting true predicates {true_predicates}")
        print_func(f"Expecting true arguments  {true_arguments}")
        
        pred_predicates, pred_arguments = y_pred[i]

        print_func(f"Got predicted predicates  {pred_predicates}")
        print_func(f"Got predicted arguments   {pred_arguments}")
        
        print_func("-"*60)
        
        for true_pred_idx, true_predicate in true_predicates.items():
            if true_pred_idx in pred_predicates:
                print_func(f"Matched predicate {true_pred_idx} = {true_predicate}")
                
                true = true_arguments[true_pred_idx]
                pred_arguments_i = pred_arguments[true_pred_idx]
                
                true_arguments_i = []
                
                for idx, true_argument in enumerate(true):
                    if true_argument['tag'] in roleset and true_argument['postag'] in ARGUMENT_POSTAGS:
                        true_arguments_i.append(true[idx])
                        
                if true_arguments_i:
                    print_func(f"Expecting arguments  {true_arguments_i}")
                    print_func(f"Got predicted        {pred_arguments_i}")
                    print_func(f"Predicted Condition Positive = {len(pred_arguments_i)}")
                    print_func(f"Condition Positive           = {len(true_arguments_i)}")
                    condition_positive += len(true_arguments_i)
                    condition_positive_i = len(true_arguments_i)
                    predicted_condition_positive += len(pred_arguments_i)

                    true_positive_i = 0

                    error_report = {
                        'example_idx' : i,
                        'predicate': true_predicate,
                        'true_arguments' : true_arguments_i,
                        'predicted_arguments': pred_arguments_i
                    }

                    for j, obj in enumerate(true_arguments_i):
                        true_tag = obj['tag']
                        true_lemma = obj['lemma']
                        true_idx = obj['idx']
                        for obj_pred in pred_arguments_i:
                            if idxmatching:
                                if obj_pred['idx'] == true_idx:
                                    true_positive_i += 1
                            else:
                                if obj_pred['idx'] == true_idx and obj_pred['tag'] == true_tag:
                                    true_positive_i += 1

                    print_func(f"True Positive = {true_positive_i}")
                    if true_positive_i != condition_positive_i:
                        error_examples.append(error_report)

                    true_positive += true_positive_i
                
        print_func("="*60)
       
    recall = true_positive/condition_positive
    precision = true_positive/predicted_condition_positive
    
    return {
        'recall': recall,
        'precision': precision,
        'f1': 2 * ((precision*recall)/(precision+recall)),
        'errors': error_examples
    }

In [8]:
#corpus_name = 'annotated_corpus_fixed+syntaxnet.json'
corpus_path = '../../data/cleared_corpus.json'
with open(corpus_path, 'r', encoding='utf-8') as f:
    corpus = json.load(f)

In [9]:
import isanlp

In [10]:
! pip install -U git+https://github.com/tchewik/isanlp_srl_framebank.git

  Cloning https://github.com/tchewik/isanlp_srl_framebank.git to /tmp/pip-req-build-bv2jg62j
  Running command git clone -q https://github.com/tchewik/isanlp_srl_framebank.git /tmp/pip-req-build-bv2jg62j
  Running command git submodule update --init --recursive -q
  Created wheel for isanl-srl-framebank: filename=isanl_srl_framebank-0.0.1-cp36-none-any.whl size=13223 sha256=0d1570c165fc47d91ec1664d4b89a39d09a317ee9c0b72bb87daba96cbde86f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-8u019tq0/wheels/53/85/ca/d8697e680129d7344263e11de6f9c43c55721e83038ff90d95
Successfully built isanl-srl-framebank


### Metrics of argument extraction quality

In [11]:
host = 'vmh2.isa.ru'
host2 = 'echistova.isa.ru'
host3 = 'tsa05.isa.ru'

ppl = PipelineDefault(address_morph=(host, 4333),
                       address_syntax=(host, 4334),
                       address_srl=(host3, 4335))

In [20]:
ppl('Мама мыла раму.')

{'text': 'Мама мыла раму.',
 'tokens': [<isanlp.annotation.Token at 0x7f2148c55c50>,
 'sentences': [<isanlp.annotation.Sentence at 0x7f2148c55c18>],
 'mystem_postag': [['S,жен,од=им,ед',
   'V,несов,пе=прош,ед,изъяв,жен',
   'S,жен,неод=вин,ед',
   '']],
 'lemma': [['мама', 'мыть', 'рама', '.']],
 'syntax_dep_tree': [[<isanlp.annotation.WordSynt at 0x7f21490b5208>,
 'morph': [[{'fPOS': 'NOUN',
    'Gender': 'Fem',
    'Animacy': 'Anim',
    'Case': 'Nom',
    'Number': 'Sing'},
   {'fPOS': 'VERB',
    'Aspect': 'Imp',
    'Valency': 'TR',
    'Tense': 'Past',
    'Number': 'Sing',
    'VerbForm': 'Fin',
    'Gender': 'Fem'},
   {'fPOS': 'NOUN',
    'Gender': 'Fem',
    'Animacy': 'Inan',
    'Case': 'Acc',
    'Number': 'Sing'},
   {}]],
 'postag': [['NOUN', 'VERB', 'NOUN', '']],
 'srl': [[<isanlp.annotation.Event at 0x7f2148c5b630>]]}

In [13]:
res = ppl('- И за это тебя посадят, -- как бы сообразив, прервала его тётя Катя.')

for event in res['srl'][0]:
    for i in range(len(event.args)):
        print(event.args[i].begin, f"({res['lemma'][0][event.args[i].begin]})", event.args[i].tag)

3 (это) цель
4 (ты) пациенс
13 (он) пациенс
15 (катя) говорящий


In [15]:
res = ppl('- И за это тебя посадят, -- как бы сообразив, прервала его тётя Катя.')

for event in res['srl'][0]:
    for i in range(len(event.args)):
        print(event.args[i].begin, f"({res['lemma'][0][event.args[i].begin]})", event.args[i].tag)

3 (это) цель
4 (ты) пациенс
13 (он) пациенс
15 (катя) говорящий


In [14]:
! pip install git+https://github.com/IINemo/isanlp.git@dev

  Cloning https://github.com/IINemo/isanlp.git (to revision dev) to /tmp/pip-req-build-vdscjvef
  Running command git clone -q https://github.com/IINemo/isanlp.git /tmp/pip-req-build-vdscjvef
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  Branch dev set up to track remote branch dev from origin.
  Created wheel for isanlp: filename=isanlp-0.0.6-cp36-none-any.whl size=33540 sha256=385dc111a08506b97664aa578f99a6e77d8512014502908d4cb87b15bb22a45f
  Stored in directory: /tmp/pip-ephem-wheel-cache-c60l3jrn/wheels/c2/22/fa/c2380ddd40a8fe7980f3ead4d7cbad4889975055f3bea6c15a
Successfully built isanlp


n_samples=400 (usually)

In [21]:
true_roles, pred_roles, texts = random_predictions(corpus, ppl, n_samples=400)

In [22]:
log_path = 'log_idx.txt'
results = compute_metrics(y_pred=pred_roles, y_true=true_roles, 
                          report_to=open(log_path, 'w', encoding='utf-8'), 
                          idxmatching=True)

copyres = dict(results)
del copyres['errors']
print_(copyres)

{'f1': 0.7944444444444444,
 'precision': 0.7447916666666666,
 'recall': 0.8511904761904762}


In [23]:
log_path = 'log_idx.txt'
results = compute_metrics(y_pred=pred_roles, y_true=true_roles, 
                          report_to=open(log_path, 'w', encoding='utf-8'), 
                          idxmatching=False)

copyres = dict(results)
del copyres['errors']
print_(copyres)

{'f1': 0.6000000000000001, 'precision': 0.5625, 'recall': 0.6428571428571429}


#### Errors 

In [25]:
import pandas as pd

ff = pd.DataFrame(results['errors'])

In [26]:
def neatify(text):
    return text.replace(' ,', ',')\
                .replace(' .', '.')\
                .replace(' ) ', ') ')\
                .replace(' ( ', ' (')\
                .replace(' : ', ': ')
                
ff['text'] = ff.example_idx.map(lambda idx: neatify(texts[idx]))

In [27]:
pd.set_option('display.max_colwidth', -1)

In [28]:
ff

,example_idx,predicate,predicted_arguments,true_arguments,text
0,1,{'lemma': 'гладить'},"[{'tag': 'тема', 'lemma': 'ладонь', 'idx': 9}, {'tag': 'агенс', 'lemma': 'он', 'idx': 0}]","[{'lemma': 'он', 'tag': 'агенс', 'idx': 0, 'postag': 'PRON'}, {'lemma': 'ее', 'tag': 'тема', 'idx': 8, 'postag': 'PRON'}]","Он взял её за руку и стал гладить её ладонь, потом стал гладить её по плечу, коснулся шеи, затылка."
1,9,{'lemma': 'признать'},"[{'tag': 'говорящий', 'lemma': 'я', 'idx': 5}, {'tag': 'пациенс', 'lemma': 'сидней', 'idx': 8}, {'tag': 'адресат', 'lemma': 'свой', 'idx': 9}]","[{'lemma': 'я', 'tag': 'говорящий', 'idx': 5, 'postag': 'PRON'}, {'lemma': 'Сидней', 'tag': 'предмет высказывания', 'idx': 8, 'postag': 'NOUN'}, {'lemma': 'свой', 'tag': 'содержание высказывания', 'idx': 9, 'postag': 'PRON'}]","Человек городской, питерский, я сразу признал Сидней своим. Это город, что называется, с головы до пят ; на его улицах, в порту среди докеров, в кварталах Ву-ла-Мулла"
2,24,{'lemma': 'выступить'},"[{'tag': 'пациенс', 'lemma': 'партия', 'idx': 24}]","[{'lemma': 'партия', 'tag': 'говорящий', 'idx': 24, 'postag': 'NOUN'}, {'lemma': 'послание', 'tag': 'содержание высказывания', 'idx': 27, 'postag': 'NOUN'}]","С определёнными оговорками можно сказать, что развитие партийной системы в России во многом зависит от того, насколько успешно выступят "" старые "" партии с обновлённым посланием к избирателю."
3,25,{'lemma': 'глядеть'},"[{'tag': 'субъект восприятия', 'lemma': 'друг', 'idx': 6}]","[{'lemma': 'мы', 'tag': 'субъект восприятия', 'idx': 0, 'postag': 'PRON'}]","Мы стояли молча, не глядя друг на друга, стояли и просто ждали, даже не подозревая, что ровно через два часа начнутся такие удивительные события, события, которые потрясут не только весь наш класс, но и всю школу..."
4,29,{'lemma': 'оценить'},"[{'tag': 'содержание мысли', 'lemma': 'уникальность', 'idx': 25}, {'tag': 'агенс', 'lemma': 'человек', 'idx': 4}]","[{'lemma': 'человек', 'tag': 'субъект психологического состояния', 'idx': 4, 'postag': 'NOUN'}, {'lemma': 'уникальность', 'tag': 'причина', 'idx': 25, 'postag': 'NOUN'}]","в том, что люди часто пытаются повторить прошлый успех, выполняя работу абсолютно таким же образом, как и раньше, не пытаясь оценить уникальность новой ситуации. На самом деле, если кто-то раньше успешно выполнил определённую работу, вряд ли он повторит этот результат снова"
5,41,{'lemma': 'подобрать'},"[{'tag': 'агенс', 'lemma': 'маруся', 'idx': 0}, {'tag': 'тема', 'lemma': 'ожерелие', 'idx': 4}]","[{'lemma': 'Маруся', 'tag': 'конечный посессор', 'idx': 0, 'postag': 'NOUN'}, {'lemma': 'ожерелье', 'tag': 'пациенс', 'idx': 4, 'postag': 'NOUN'}]",Маруся подобрала шелестящее ракушечное ожерелье Эле-Фантика и побрела домой.
6,44,{'lemma': 'гудеть'},"[{'tag': 'источник звука', 'lemma': 'мой', 'idx': 67}]","[{'lemma': 'кровь', 'tag': 'источник звука', 'idx': 66, 'postag': 'NOUN'}]","Я стал ловить девочкин взгляд, хотя никто в мире еще не поймал взгляда младенца, но я очень старался, я понимал, что эта безграничная ласка младенческого взгляда -- к миру вообще, жемчужно - серому безбрежному миру ; он не различает еще отдельностей, только смутные тени света, волны тепла, к тому же я не знал, где я сам, кровь моя гудела, было уже больно от этого... но если девочка уловит зрачки мои, то мы увидимся наконец !"
7,46,{'lemma': 'принудить'},"[{'tag': 'пациенс', 'lemma': 'свой', 'idx': 16}]","[{'lemma': 'генерал', 'tag': 'пациенс', 'idx': 0, 'postag': 'NOUN'}]","Генерал встретил нас изъявлением искреннего своего сожаления о происшедшем и о том, что против хотения своего принуждён теперь лишиться нас."
8,49,{'lemma': 'жить'},"[{'tag': 'содержание действия', 'lemma': 'стандарт', 'idx': 8}, {'tag': 'срок', 'lemma': 'пора', 'idx': 11}, {'tag': 'тема', 'lemma': 'образ', 'idx': 4}]","[{'lemma': 'образ', 'tag': 'тема', 'idx': 4, 'postag': 'NOUN'}, {'lemma': 'стандарт', 'tag': 'сфера', 'idx': 8, 'postag': 'NOUN'}, {'lemma': 'образ', 'tag': 'содержание мысли', 'idx': 13, 'postag': 'NOU